In [26]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import time
from datetime import datetime
from tensorflow import math
from tensorflow import reduce_mean
import tensorflow as tf

In [27]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    print('X: \n', X[0:5], '\n Y: \n', y[0:5])
    return asarray(X), asarray(y)

In [3]:
# load the dataset
df = read_csv("SolarTrainingData.csv", index_col=0)

In [4]:
# try to drop the day column for better results
df.tail()

temp  feelslike   dew  humidity  precip  snow  snowdepth  windgust  \
73657  97.4       99.1  62.8     32.19     0.0     0          0      16.1   
73660  97.4       99.1  62.8     32.19     0.0     0          0      16.1   
73663  97.4       99.1  62.8     32.19     0.0     0          0      16.1   
73664  97.4       99.1  62.8     32.19     0.0     0          0      16.1   
73666  97.4       99.1  62.8     32.19     0.0     0          0      16.1   

       windspeed  winddir  ...  cloudcover  visibility  solarradiation  \
73657       10.6    138.0  ...        11.3         9.9           405.0   
73660       10.6    138.0  ...        11.3         9.9           405.0   
73663       10.6    138.0  ...        11.3         9.9           405.0   
73664       10.6    138.0  ...        11.3         9.9           405.0   
73666       10.6    138.0  ...        11.3         9.9           405.0   

       solarenergy  uvindex  year  month  day  time  Percent Output  
73657          1.5      4.0  2018      8   31  18.0        0.106900  
73660          1.5      4.0  2018      8   31  18.0        0.295373  
73663          1.5      4.0  2018      8   31  18.0        0.022917  
73664          1.5      4.0  2018      8   31  18.0        0.346222  
73666          1.5      4.0  2018      8   31  18.0        0.236087  

[5 rows x 21 columns]

In [13]:
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 21))
# split into train/test
n_test = 100
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X: 
 [array([[2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000e+00,
        2.0180000e+03, 1.0000000e+00, 1.0000000e+00, 8.0000000e+00,
        1.4920635e-02],
       [2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000e+00,
        2.0180000e+03, 1.0000000e+00, 1.0000000e+00, 8.0000000e+00,
        1.5855573e-02],
       [2.5000000e+01, 1.4100000e+01, 1.1400000e+01, 5.5860001e+01,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8900000e+01,
        1.1700000e+01, 1.0000000e+01, 1.0416000e+03, 9.7500000e+01,
        9.8999996e+00, 1.8000000e+01, 1.0000000e-01, 0.0000000

In [32]:
# define model
# improvement area : try adding dropout
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,21)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss=custom_eval, metrics=['mae'])

In [33]:
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

Epoch 1/350
(None, 21)


NotImplementedError: in user code:

    File "C:\Users\trevo\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\trevo\AppData\Local\Temp\ipykernel_29432\1684113005.py", line 6, in custom_eval  *
        RMSE = sqrt(MSE)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (custom_eval/Mean:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.


In [31]:
# Need new function to evaluate RMSE based on just 'Percent Output'
# Current state: we have input tensors of shape (None, 21). Need to find a way to turn that into an "array of losses"
# of shape (None, 21). Then return that.
def custom_eval(y_actual, y_pred):
    print(y_actual.get_shape())
    SE_Tensor = math.squared_difference(y_pred, y_actual)  #squared difference
    MSE = reduce_mean(SE_Tensor)
    RMSE = sqrt(MSE)
    return RMSE

In [15]:
# New loss function based on just 'Percent Output'
def custom_loss(y_actual, y_predicted):
    print(type(y_predicted))
    print(y_predicted)
    return 1

In [50]:
# Testing this from sample
def my_loss_fn(y_true, y_pred):
    print(type(y_true))
    return 1

In [48]:
print(type(y_test))

<class 'numpy.ndarray'>


In [ ]:
# make a prediction
#row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
#yhat = model.predict(row)
#print('Predicted: %.3f' % (yhat))